# Merging of ``DataFrames``

In [1]:
from pandas import DataFrame
from viedoors import CADLoader, NPALoader, FileMerger, HMLoader
from viedoors import BSTLoader, FLTLoader, FMLoader

In [2]:
cad = CADLoader(file="data/420_gesamt_20250122.xlsx", title="CAD")
npa = NPALoader(file="data/NPA_Tuer Aufnahme Obj420.xlsx", title="NPA")
hm = HMLoader(file="data/Schrack_HM_Obj420.xls", title="HM")
bst = BSTLoader(file="data/Sisando_BST_Obj420.xlsx", title="BST")
flt = FLTLoader(file="data/Sisando_FLT_Obj420.xlsx", title="FLT")
fm = FMLoader(file="data/BSTüren_Filemaker_11.04.22.xlsx", title="FM")

df_npa = npa.get_data(prefixed=True)
df_cad = cad.get_data(prefixed=True)
df_hm = hm.get_data(prefixed=True)
df_bst = bst.get_data(prefixed=True)
df_flt = flt.get_data(prefixed=True)
df_fm = fm.get_data(prefixed=True)

merger = FileMerger(files=[df_cad, df_npa, df_hm, df_bst, df_flt, df_fm], how="left")
merge = merger.get_data_merge()

In [3]:
merge.to_excel("matching/420_match_file.xlsx")

In [4]:
merger.merge_success_rate()

0.06663560111835974

In [8]:
merger.find_non_matching_rows()

AssertionError: This function can only be used when merging two datafiles. You tried to merge 6.

In [6]:

for dataset in [df_npa, df_hm, df_bst, df_flt, df_fm]:
    print("---")
    print(dataset.columns[0].split("___")[0]+"-Datenfile")

    fm = FileMerger(files=[df_cad, dataset], how="inner")

    a = len(dataset)
    b = len(fm.get_data_merge())

    print(f"Datensätze: {a}.")
    print(f"Übereinstimmungen mit CAD-Datenfile: {b} ({round(b/a*100,0)}%)")


---
NPA-Datenfile
Datensätze: 114.
Übereinstimmungen mit CAD-Datenfile: 112 (98.0%)
---
HM-Datenfile
Datensätze: 41.
Übereinstimmungen mit CAD-Datenfile: 30 (73.0%)
---
BST-Datenfile
Datensätze: 74.
Übereinstimmungen mit CAD-Datenfile: 75 (101.0%)
---
FLT-Datenfile
Datensätze: 83.
Übereinstimmungen mit CAD-Datenfile: 68 (82.0%)
---
FM-Datenfile
Datensätze: 6438.
Übereinstimmungen mit CAD-Datenfile: 29 (0.0%)


In [7]:

for dataset in [df_npa, df_hm, df_bst, df_flt, df_fm]:
    name = dataset.columns[0].split("___")[0]+"-Datenfile"
    print("---")
    print(name)

    fm = FileMerger(files=[df_cad, dataset], how="inner")

    a = len(dataset)
    b = len(fm.get_data_merge())

    print(f"Datensätze: {a}.")
    print(f"Übereinstimmungen mit CAD-Datenfile: {b} ({round(b/a*100,0)}%)")

    nm = fm.find_non_matching_rows()

    nm.to_excel(f"non_matching/CAD_versus_{name}.xlsx")



---
NPA-Datenfile
Datensätze: 114.
Übereinstimmungen mit CAD-Datenfile: 112 (98.0%)
---
HM-Datenfile
Datensätze: 41.
Übereinstimmungen mit CAD-Datenfile: 30 (73.0%)
---
BST-Datenfile
Datensätze: 74.
Übereinstimmungen mit CAD-Datenfile: 75 (101.0%)
---
FLT-Datenfile
Datensätze: 83.
Übereinstimmungen mit CAD-Datenfile: 68 (82.0%)
---
FM-Datenfile
Datensätze: 6438.
Übereinstimmungen mit CAD-Datenfile: 29 (0.0%)
